In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow.keras as keras

# Function to normalize the images
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.cast(image, tf.float32) / 255.0
    # Normalizing to mean=0.5 and std=0.5
    image = (image - 0.5) / 0.5
    return image, label

# Load CIFAR-10 dataset into ./data-tf directory
cifar10_data, cifar10_info = tfds.load('cifar10', with_info=True, as_supervised=True, data_dir='./data-tf')

# Extract train and test datasets
train_data, test_data = cifar10_data['train'], cifar10_data['test']

# Normalize the datasets
train_data = train_data.map(normalize_img)
test_data = test_data.map(normalize_img)

# Batch the datasets
batch_size = 16
train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

# Display dataset information
# cifar10_info
print("TensorFlow version: ", tf.version.VERSION)
print("Keras version: ", keras.__version__)
next(iter(train_data))[1]

2024-08-30 14:32:31.583503: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-30 14:32:31.586253: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-30 14:32:31.596489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 14:32:31.609822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 14:32:31.613276: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 14:32:31.623937: I tensorflow/core/platform/cpu_feature_gu

<tf.Tensor: shape=(16,), dtype=int64, numpy=array([7, 8, 4, 4, 6, 5, 2, 9, 6, 6, 9, 9, 3, 0, 8, 7])>

The CIFAR-10 dataset has been successfully loaded using TensorFlow Datasets. The dataset contains 50,000 training examples and 10,000 test examples, each with 32x32 color images in 10 different classes.

> create network in tf without using keras:
class Net(nn.Module):
    SLOPE = 0.01
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

In [ ]:
import tensorflow as tf

class Net(tf.Module):
    SLOPE = 0.01

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = tf.Variable(tf.random.normal([5, 5, 3, 6]), name='conv1')
        self.conv2 = tf.Variable(tf.random.normal([5, 5, 6, 16]), name='conv2')
        self.fc1 = tf.Variable(tf.random.normal([16 * 5 * 5, 120]), name='fc1')
        self.fc2 = tf.Variable(tf.random.normal([120, 84]), name='fc2')
        self.fc3 = tf.Variable(tf.random.normal([84, 10]), name='fc3')
        self.pool = tf.nn.max_pool2d

    def __call__(self, x):
        x = tf.nn.conv2d(x, self.conv1, strides=[1, 1, 1, 1], padding='VALID')
        x = tf.nn.relu(x)
        x = self.pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        x = tf.nn.conv2d(x, self.conv2, strides=[1, 1, 1, 1], padding='VALID')
        x = tf.nn.relu(x)
        x = self.pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        x = tf.reshape(x, [-1, 16 * 5 * 5])
        x = tf.matmul(x, self.fc1)
        x = tf.nn.leaky_relu(x, alpha=self.SLOPE)
        x = tf.matmul(x, self.fc2)
        x = tf.nn.leaky_relu(x, alpha=self.SLOPE)
        x = tf.matmul(x, self.fc3)
        return x

# Instantiate the network
net = Net()

The convolutional neural network has been successfully created using TensorFlow without Keras. The network is instantiated and ready for use.

> write training loop input = test_data, network = net, 1 epoch, print average loss

In [ ]:
# Define the training loop and Train the network for one epoch

def train_one_epoch(dataset, model, optimizer, loss_fn):
    total_loss = 0.0
    num_batches = 0

    for images, labels in dataset:
        with tf.GradientTape() as tape:
            outputs = model(images)
            loss = loss_fn(labels, outputs)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        total_loss += loss.numpy()
        num_batches += 1

    average_loss = total_loss / num_batches
    return average_loss

# Set up the optimizer and loss function
optimizer = tf.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999)
criterion = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Train for 6 epochs
num_epochs = 6
for epoch in range(num_epochs):
    average_loss = train_one_epoch(test_data, net, optimizer, criterion)
    print(f'Epoch {epoch + 1}, Average Loss: {average_loss:.4f}')
    
# Output the final average loss
average_loss

Epoch 1, Average Loss: 1.8258
Epoch 2, Average Loss: 1.8132
2024-08-29 05:46:33.090707: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Epoch 3, Average Loss: 1.8058
Epoch 4, Average Loss: 1.8046
Epoch 5, Average Loss: 1.8032
Epoch 6, Average Loss: 1.7917


1.791662993812561

## Save trained model

In [ ]:
# Save the trained model
import tensorflow as tf

# Save the model weights
checkpoint = tf.train.Checkpoint(model=net)
checkpoint.save('cifar10_model.ckpt')

'cifar10_model.ckpt-1'

The trained model has been saved as `cifar10_model.ckpt-1`.

> load data model from 'cifar10_model.ckpt'

In [ ]:
# Load the trained model from the checkpoint
checkpoint = tf.train.Checkpoint(model=net)
checkpoint.restore('cifar10_model.ckpt-1')

# Verify that the model has been loaded correctly by running a test image through it
for image, label in test_data.take(1).cache():
    output = net(image)
    # Display output of only 1 classification
    print("Raw output:", output[0])
    # Compute the softmax of output
    softmax_output = tf.nn.softmax(output)
    print("Softmax output:", softmax_output[0])

Raw output: tf.Tensor(
[-138.09183 -137.39015 -136.01796 -135.43976 -134.64357 -135.36809
 -134.70815 -135.31839 -139.15324 -137.35419], shape=(10,), dtype=float32)
Softmax output: tf.Tensor(
[0.00834945 0.01684191 0.0664243  0.11842314 0.2625529  0.12722215
 0.2461343  0.13370456 0.00288862 0.01745865], shape=(10,), dtype=float32)
2024-08-29 05:44:46.324797: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


The model has been successfully loaded from the checkpoint and verified by running a test image through it.

> evaluate accuracy on test set per each class

In [ ]:
# Evaluate accuracy on the test set per each class

# Function to calculate accuracy per class
import numpy as np

def evaluate_accuracy_per_class(model, dataset, num_classes=10):
    class_correct = np.zeros(num_classes)
    class_total = np.zeros(num_classes)

    for images, labels in dataset:
        outputs = model(images)
        _, predicted = tf.math.top_k(outputs, 1)
        predicted = tf.squeeze(predicted, axis=1)
        correct = tf.equal(predicted, tf.cast(labels, tf.int32))

        for i in range(len(labels)):
            label = labels[i].numpy()
            class_correct[label] += correct[i].numpy()
            class_total[label] += 1

    class_accuracy = [class_correct[i] / class_total[i] if class_total[i] > 0 else 0 for i in range(num_classes)]
    return class_accuracy

# Evaluate the model on the test set
class_accuracy = evaluate_accuracy_per_class(net, test_data)
class_accuracy

[0.539, 0.388, 0.331, 0.333, 0.273, 0.239, 0.462, 0.091, 0.426, 0.557]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=96230c3c-90f8-4675-a637-f681b2b6286f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>